### Imports

In [1]:
import numpy as np
np.set_printoptions(suppress=True)

Question  d)

In [2]:
from scipy.optimize import minimize, NonlinearConstraint, LinearConstraint
from numpy import cos, pi

In [3]:
# constants
n = 10
alpha = 1.5
rmin = 1.0
rmax = 2.0
theta = 2 * pi / (5 * (n + 1))
costh = cos(theta)  # radians

In [4]:
### Linear constraints ###
lin_const = []

# rmin <= r_i <= rmax (2)
A2 = LinearConstraint(np.eye(n), rmin, rmax)
lin_const.append(A2)

# -theta*alpha <= r_{i+1} - r_i <= theta*alpha for i = 1, ..., n (4)
M = (np.eye(n+1) - np.eye(n+1, k=-1))[1:-1, :-1]

# col 1 is r1, col 2 is r2, etc.
A4 = LinearConstraint(M, -theta*alpha, theta*alpha)
lin_const.append(A4)

# i = 0
Mr1 = np.append(1, np.zeros(n-1, dtype=int))
Ar1 = LinearConstraint(Mr1, rmin-theta*alpha, rmin+theta*alpha)
lin_const.append(Ar1)

# i = n
Mrn = np.append(np.zeros(n-1, dtype=int), 1)
Arn = LinearConstraint(Mrn, rmax-theta*alpha, rmax+theta*alpha)
lin_const.append(Arn)

In [5]:
### Nonlinear constraints ###

# 2r_{i-1}*r_{i+1}*cos(theta) - r_i*r_{i-1} - r_i*r_{i+1} <= 0 for i = 0, ..., n+1 (3)
non_lin_const = []

# i = 0
nlcon_0 = NonlinearConstraint(lambda r: 2*rmin*r[0]*costh - rmin*rmin - rmin*r[0], -np.inf, 0)
non_lin_const.append(nlcon_0)

# i = 1
nlcon_1 = NonlinearConstraint(lambda r: 2*rmin*r[1]*costh - r[0]*rmin - r[0]*r[1], -np.inf, 0)
non_lin_const.append(nlcon_1)

# WARNING: we need to fix i in lambda, because otherwise it will always use the last reference value of i!!!
# i = 2...n-1
for i in range(2, n-1):
    non_lin_const.append(NonlinearConstraint(lambda r, idx=i: 2*r[idx-2]*r[idx]*costh - r[idx-1]*r[idx-2] - r[idx-1]*r[idx], -np.inf, 0))  

# i = n
nlcon_n = NonlinearConstraint(lambda r: 2*r[n-2]*rmax*costh - r[n-1]*r[n-2] - r[n-1]*rmax, -np.inf, 0)
non_lin_const.append(nlcon_n)

# i = n+1
nlcon_np1 = NonlinearConstraint(lambda r: 2*r[n-1]*r[n-1]*costh - rmax*r[n-1] - rmax*r[n-1], -np.inf, 0)
non_lin_const.append(nlcon_np1)

In [6]:
### Objective function ###
# maximize sum of r_i
def obj(r):
    return -np.mean(r)

In [7]:
# initial design (r0)
r0 = rmin*np.ones(n)

### Solve ###
res = minimize(obj, r0,
    method="trust-constr",
    constraints=[*lin_const, *non_lin_const],
    options={"maxiter": 10000}
)

# print results
print(res)

 barrier_parameter: 3.200000000000001e-05
 barrier_tolerance: 3.200000000000001e-05
          cg_niter: 40
      cg_stop_cond: 4
            constr: [array([1.01320412, 1.0405017 , 1.08382279, 1.14649342, 1.23391615,
       1.35484973, 1.52399642, 1.69516386, 1.8661072 , 1.99981608]), array([0.02729758, 0.04332109, 0.06267063, 0.08742273, 0.12093358,
       0.16914668, 0.17116744, 0.17094334, 0.13370888]), array([1.01320412]), array([1.99981608]), array([-0.00000454]), array([-0.00000586]), array([-0.00000849]), array([-0.00001009]), array([-0.00001391]), array([-0.00004509]), array([-0.0001045]), array([-0.08476608]), array([-0.09597488]), array([-0.31572967]), array([-0.05287215])]
       constr_nfev: [0, 0, 0, 0, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286]
       constr_nhev: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
       constr_njev: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 0.16112589836120605

In [8]:
# print results
print(np.round(res.x,4))

[1.0132 1.0405 1.0838 1.1465 1.2339 1.3548 1.524  1.6952 1.8661 1.9998]


`[1.0132 1.0405 1.0839 1.1466 1.2341 1.3551 1.5246 1.6959 1.8673 2.0000]`